In [1]:
import rawpy
import cv2
import numpy as np

# Set resize scale factor
scaleFactor = 0.2
filterKernelSize = 5

# Set Image path in array
pathRawImage = [
    "raw image/raw image 1.NEF",
    "raw image/raw image 2.NEF",
    "raw image/raw image 3.NEF",
    "raw image/raw image 4.NEF",
]
pathGroundTruthImage = [
    "ground truth image/ground truth image 1.tiff",
    "ground truth image/ground truth image 2.tiff",
    "ground truth image/ground truth image 3.tiff",
    "ground truth image/ground truth image 4.tiff",
]

In [19]:
# Functions ***********************************************************************************************************


# Converter +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
class Converter:
    # Convert Raw to BGR
    def raw2bgr(imgSet, cameraWB=True):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.cvtColor(
                rawpy.imread(imgSet).postprocess(use_camera_wb=cameraWB),
                cv2.COLOR_RGB2BGR,
            )
        else:
            return [
                cv2.cvtColor(
                    rawpy.imread(img).postprocess(use_camera_wb=cameraWB),
                    cv2.COLOR_RGB2BGR,
                )
                for img in imgSet
            ]

    # Convert RGB to Gray
    def rgb2gray(imgSet):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.cvtColor(imgSet, cv2.COLOR_RGB2GRAY)
        else:
            return [cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in imgSet]

    # Convert RGB to BGR
    def rgb2bgr(imgSet):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.cvtColor(imgSet, cv2.COLOR_RGB2BGR)
        else:
            return [cv2.cvtColor(img, cv2.COLOR_RGB2BGR) for img in imgSet]

    # Convert BGR to Gray
    def bgr2gray(imgSet):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.cvtColor(imgSet, cv2.COLOR_BGR2GRAY)
        else:
            return [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in imgSet]

    # Convert BGR to RGB
    def bgr2rgb(imgSet):
        # Convert BGR image to RGB
        return [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in imgSet]

    # Convert Gray to BGR
    def gray2bgr(imgSet):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.cvtColor(imgSet, cv2.COLOR_GRAY2BGR)
        else:
            return [cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) for img in imgSet]


# Manipulate image ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
class Image:
    # Read image set
    def read(imgSet):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.imread(imgSet)
        else:
            return [cv2.imread(img) for img in imgSet]

    # Save image to TIFF
    def saveTIFF(imgSet, fileName):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            cv2.imwrite(fileName, imgSet)
        elif len(imgSet) != len(fileName):
            print("Error: Image length mismatch")
        else:
            for i in range(len(imgSet)):
                cv2.imwrite(fileName[i], imgSet[i])
        return None

    # Scale images
    def scale(imgSet, scaleFactor=0.1):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.resize(imgSet, None, fx=scaleFactor, fy=scaleFactor)
        else:
            return [
                cv2.resize(img, None, fx=scaleFactor, fy=scaleFactor) for img in imgSet
            ]

    # Concatenate images
    def concatenate(imgSet1, imgSet2, axis=1):
        # Concatenate two images, axis=1 for horizontal, axis=0 for vertical
        # Check if imgSet is array or not
        if not isinstance(imgSet1, list) and not isinstance(imgSet2, list):
            return np.concatenate((imgSet1, imgSet2), axis=axis)
        elif not isinstance(imgSet1, list) or not isinstance(imgSet2, list):
            print("Error: Image type mismatch")
        elif len(imgSet1) != len(imgSet2):
            print("Error: Image length mismatch")
        else:
            # Concatenate images
            concatImg = [None] * len(imgSet1)
            for i in range(len(imgSet1)):
                # Check image size
                if imgSet1[i].shape[0] != imgSet2[i].shape[0]:
                    print("Error: Image size mismatch at image " + str(i + 1))
                    return None
                else:
                    concatImg[i] = np.concatenate((imgSet1[i], imgSet2[i]), axis=axis)

            return concatImg
        return None

    # Show all images
    def show(imgSet, name="image"):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            cv2.imshow(name, imgSet)
            return None
        elif len(imgSet) == 0:
            print("Error: Image set is empty")
            return None
        else:
            for i in range(len(imgSet)):
                fileName = pathRawImage[i].split("/")[-1]
                cv2.imshow("Image " + str(i + 1) + ': "' + fileName + '"', imgSet[i])


# Smooth Filter +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# 1. Average Filter
# 2. Box Filter
# 3. Gaussian Filter
# 4. Median Filter
# 5. Bilateral Filter
# 6. Non-Local Means Filter
# 7. Custom Kernel Filter
class SmoothFilter:

    def applyAverageFilter(imgSet, kernelSizeX=5, kernelSizeY=5):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.blur(imgSet, (kernelSizeX, kernelSizeY))
        else:
            return [cv2.blur(img, (kernelSizeX, kernelSizeY)) for img in imgSet]

    def applyBoxFilter(imgSet, kernelSizeX=5, kernelSizeY=5):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.boxFilter(imgSet, -1, (kernelSizeX, kernelSizeY))
        else:
            return [
                cv2.boxFilter(img, -1, (kernelSizeX, kernelSizeY)) for img in imgSet
            ]

    def applyGaussianBlurFilter(imgSet, kernelSizeX=5, kernelSizeY=5):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.GaussianBlur(imgSet, (kernelSizeX, kernelSizeY), 0)
        else:
            return [
                cv2.GaussianBlur(img, (kernelSizeX, kernelSizeY), 0) for img in imgSet
            ]

    def applyMedianBlurFilter(imgSet, kernelSize=5):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.medianBlur(imgSet, kernelSize)
        else:
            return [cv2.medianBlur(img, kernelSize) for img in imgSet]

    def applyBilateralFilter(imgSet, d=9, sigmaColor=75, sigmaSpace=75):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.bilateralFilter(imgSet, d, sigmaColor, sigmaSpace)
        else:
            return [
                cv2.bilateralFilter(img, d, sigmaColor, sigmaSpace) for img in imgSet
            ]

    def applyNonLocalMeansFilter(imgSet, h=10, searchWindowSize=20):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.fastNlMeansDenoising(imgSet, None, h, searchWindowSize)
        else:
            return [
                cv2.fastNlMeansDenoising(img, None, h, searchWindowSize)
                for img in imgSet
            ]

    def applyCustomKernelFilter(imgSet, kSize):
        kernel = np.ones((kSize, kSize), np.float32) / (kSize * kSize)
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.filter2D(imgSet, -1, kernel)
        else:
            return [cv2.filter2D(img, -1, kernel) for img in imgSet]


# Edge Detection Filter ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# 1. Canny Edge Detection
# 2. Sobel Edge Detection
# 3. Laplacian Edge Detection
# 4. Scharr Edge Detection
# 5. Prewitt Edge Detection
class EdgeDetectionFilter:
    def applyCannyEdgeDetection(imgSet, threshold1=100, threshold2=200):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.Canny(imgSet, threshold1, threshold2)
        else:
            return [cv2.Canny(img, threshold1, threshold2) for img in imgSet]

    def applySobelEdgeDetection(imgSet, ddepth=-1, dx=1, dy=1, ksize=5):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.Sobel(imgSet, ddepth, dx, dy, ksize)
        else:
            return [cv2.Sobel(img, ddepth, dx, dy, ksize) for img in imgSet]

    def applyLaplacianEdgeDetection(imgSet, ddepth=-1):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.Laplacian(imgSet, ddepth)
        else:
            return [cv2.Laplacian(img, ddepth) for img in imgSet]

    def applyScharrEdgeDetection(imgSet, ddepth=-1, dx=1, dy=0):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.Scharr(imgSet, ddepth, dx, dy)
        else:
            return [cv2.Scharr(img, ddepth, dx, dy) for img in imgSet]

    def applyPrewittEdgeDetection(imgSet, ddepth=-1, dx=1, dy=1):
        # Check if image is array or not
        if not isinstance(imgSet, list):
            return cv2.Sobel(imgSet, ddepth, dx, dy, ksize=3)
        else:
            return [cv2.Sobel(img, ddepth, dx, dy, ksize=3) for img in imgSet]


class Compute:
    def peakSignalToNoiseRatio(img1, img2, decimal=4):
        # Check if image is array or not
        if not isinstance(img1, list) and not isinstance(img2, list):
            return cv2.PSNR(img1, img2)
        elif not isinstance(img1, list) or not isinstance(img2, list):
            print("Error: Image type mismatch")
        elif len(img1) != len(img2):
            print("Error: Image length mismatch")
        else:
            # Compute PSNR
            psnr = [None] * len(img1)
            for i in range(len(img1)):
                psnr[i] = round(cv2.PSNR(img1[i], img2[i]), decimal)

            return psnr
        return None

    def compressionRatio(img1, img2, decimal=4):
        # Check if image is array or not
        if not isinstance(img1, list) and not isinstance(img2, list):
            return round((img1.nbytes / img2.nbytes), decimal)
        elif not isinstance(img1, list) or not isinstance(img2, list):
            print("Error: Image type mismatch")
        elif len(img1) != len(img2):
            print("Error: Image length mismatch")
        else:
            # Compute Compression Ratio
            cr = [None] * len(img1)
            for i in range(len(img1)):
                cr[i] = round((img1[i].nbytes / img2[i].nbytes), decimal)

            return cr
        return None

In [3]:
# Step 1: Reading RAW Image and converting to BGR for OpenCV (Read RAW Image) ===============================# Get Processed Image
rawImageSet = Converter.raw2bgr(pathRawImage)
rawScaledImageSet = Image.scale(rawImageSet, scaleFactor)

In [4]:
# Step 2: Pre-process image, create "Ground Truth Image" (Grayscale, Resize, Save) ==========================
grayImageSet = Converter.bgr2gray(rawImageSet)
grayResizedImagesSet = Image.scale(grayImageSet, scaleFactor)

# Save the processed image object to tagged image file TIFF as ground truth image
# saveTIFF(grayResizedImagesSet, pathGroundTruthImage)

In [23]:
# Step 3: Apply edge detection ==============================================================================
# Step 3.1: Read Ground Truth Image
groundTruthImageSet = Image.read(pathGroundTruthImage)

# Step 3.2: Apply smoothing filter to the image
# Average filter
averageFilteredSet = SmoothFilter.applyAverageFilter(
    groundTruthImageSet, filterKernelSize, filterKernelSize
)
# Box filter
boxFilteredSet = SmoothFilter.applyBoxFilter(
    groundTruthImageSet, filterKernelSize, filterKernelSize
)
# Gaussian filter
gaussianFilteredSet = SmoothFilter.applyGaussianBlurFilter(
    groundTruthImageSet, filterKernelSize, filterKernelSize
)
# Median filter
medianFilteredSet = SmoothFilter.applyMedianBlurFilter(
    groundTruthImageSet, filterKernelSize
)
# Bilateral filter
bilateralFilteredSet = SmoothFilter.applyBilateralFilter(groundTruthImageSet, 9, 75, 75)
# Non-Local Means filter
nonLocalMeansFilteredSet = SmoothFilter.applyNonLocalMeansFilter(
    groundTruthImageSet, 10, 20
)
# Custom kernel filter
customKernelFilteredSet = SmoothFilter.applyCustomKernelFilter(
    groundTruthImageSet, filterKernelSize
)
# Compile all filtered set
smoothFilteredSet = [
    averageFilteredSet,
    boxFilteredSet,
    gaussianFilteredSet,
    medianFilteredSet,
    bilateralFilteredSet,
    nonLocalMeansFilteredSet,
    customKernelFilteredSet,
]

# Step 3.3: Apply edge detection
# Canny edge detection
# TODO: Fix Canny edge detection
# cannyEdgeDetectedSet = EdgeDetectionFilter.applyCannyEdgeDetection(
# smoothFilteredSet, 100, 200
# )
# Sobel edge detection
# TODO: Fix Sobel edge detection
# sobelEdgeDetectedSet = EdgeDetectionFilter.applySobelEdgeDetection(smoothFilteredSet)
# Laplacian edge detection
# TODO: Fix Laplacian edge detection
# laplacianEdgeDetectedSet = EdgeDetectionFilter.applyLaplacianEdgeDetection(
#     smoothFilteredSet
# )
# Scharr edge detection
# TODO: Fix Scharr edge detection
# scharrEdgeDetectedSet = EdgeDetectionFilter.applyScharrEdgeDetection(smoothFilteredSet)
# Prewitt edge detection
# TODO: Fix Prewitt edge detection
# prewittEdgeDetectedSet = EdgeDetectionFilter.applyPrewittEdgeDetection(
#     smoothFilteredSet
# )
# Compile all edge detection set
# edgeDetectedSet = [
#     cannyEdgeDetectedSet,
#     sobelEdgeDetectedSet,
#     laplacianEdgeDetectedSet,
#     scharrEdgeDetectedSet,
#     prewittEdgeDetectedSet,
# ]

In [17]:
# Step 4 Compute Peak Signal-to-Noise Ratio (PSNR), compression ratio =======================================
# Compute PSNR between ground truth image and all types of smoothing filter
psnr = [
    Compute.peakSignalToNoiseRatio(groundTruthImageSet, imgSet, 2)
    for imgSet in smoothFilteredSet
]

# Compute Compression Ratio between ground truth image and all types of smoothing filter
cr = [
    Compute.compressionRatio(groundTruthImageSet, imgSet, 2)
    for imgSet in smoothFilteredSet
]

for i in range(len(psnr)):
    print("PSNR for smooth filter", i + 1, ":", psnr[i])

for i in range(len(cr)):
    print("Compression Ratio for smooth filter", i + 1, ":", cr[i])

PSNR for smooth filter 1 : [25.39, 27.17, 23.89, 37.82]
PSNR for smooth filter 2 : [25.39, 27.17, 23.89, 37.82]
PSNR for smooth filter 3 : [27.85, 30.49, 26.56, 40.78]
PSNR for smooth filter 4 : [26.31, 27.94, 24.6, 42.03]
PSNR for smooth filter 5 : [32.06, 35.72, 34.77, 42.52]
PSNR for smooth filter 6 : [34.52, 35.1, 35.26, 41.14]
PSNR for smooth filter 7 : [25.39, 27.17, 23.89, 37.82]


In [25]:
# set1 = scaleImages(groundTruthImageSet, scaleFactor)
# set2 = scaleImages(Converter.gray2bgr(groundTruthImageSet), scaleFactor)
# rawXgroundTruthSet = concatImages(set1, set2)
# gtiXfilteredSet = concatenate(groundTruthImageSet, smoothFilteredSet[2])

# Image.showImages(rawImageSet)
# Image.showImages(groundTruthImageSet)
# Image.showImages(grayImageSet)
# Image.showImages(grayResizedImagesSet)
# Image.showImages(rawXgroundTruthSet, "Raw Image X Ground Truth Image")

# Smooth Filtered
# Image.show(smoothFilteredSet[0], "Smooth Filtered Image")  # Average Filtered Image
# Image.show(smoothFilteredSet[1], "Smooth Filtered Image") # Box Filtered Image
# Image.show(smoothFilteredSet[2], "Smooth Filtered Image") # Gaussian Filtered Image
# Image.show(smoothFilteredSet[3], "Smooth Filtered Image") # Median Filtered Image
# Image.show(smoothFilteredSet[4], "Smooth Filtered Image") # Bilateral Filtered Image
# Image.show(smoothFilteredSet[5], "Smooth Filtered Image") # Non-Local Means Filtered Image
# Image.show(smoothFilteredSet[6], "Smooth Filtered Image") # Custom Kernel Filtered Image

# Edge Detected Filtered
# Image.show(edgeDetectedSet, "Edge Detected Filtered Image")

# Concatenated
# Image.show(gtiXfilteredSet, "Average Filtered Image")

cv2.waitKey(0)
cv2.destroyAllWindows()